Arabidopsis seedlings are incubated with certain pharmaceuticals that attack the cell wall of the plants. This leads to increased production of lignin, a polymer that causes the seedlings to "stiffen" so that nothing can get through the cell wall anymore. For analysis, we use a specific dye that docks onto the individual lignin polymers, making them visible. And this is exactly where our interest lies. Using Fiji or ImageJ, we have to manually quantify the stained regions for every sample we analyse, sometimes several hundreds in number. Instead, we would like to have a tool that can be fed with all the images and quantifies the stained regions within seconds.

<img src="images/root0001.png" width="400" height="200">

In [34]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import napari
import sys

In [11]:
# Read image
img = cv.imread("images/root0002.tif")

In [3]:
# Display image
cv.imshow("image", img)
cv.waitKey(0)
cv.destroyAllWindows()

In [17]:
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
cv.imshow("grayscale image", img_gray)
cv.waitKey(0)
cv.destroyAllWindows()

In [21]:
viewer = napari.Viewer()
viewer.add_image(img, rgb=True)
viewer.add_image(img_gray, rgb=False)

<Image layer 'img_gray' at 0x27700065510>

In [41]:
np.set_printoptions(threshold=sys.maxsize)

[x, y] = img_gray.shape
max_row = np.zeros(x)
min_row = np.zeros(x)

for i in range(x):
    max_row[i] = np.max(img_gray[i,:])
    min_row[i] = np.min(img_gray[i,:])

overall_max = np.max(max_row)
overall_min = np.min(min_row)

print(max_row, min_row, overall_max, overall_min)

[190. 190. 188. 187. 188. 188. 187. 190. 189. 187. 185. 188. 188. 191.
 190. 187. 187. 188. 190. 186. 185. 189. 189. 187. 185. 185. 186. 186.
 186. 185. 185. 185. 186. 191. 187. 185. 185. 191. 187. 187. 184. 186.
 185. 185. 188. 189. 185. 188. 185. 188. 193. 188. 187. 185. 186. 187.
 192. 190. 188. 187. 186. 187. 189. 188. 186. 187. 187. 186. 187. 185.
 189. 187. 186. 187. 187. 192. 187. 186. 192. 188. 188. 187. 193. 185.
 192. 190. 190. 187. 189. 189. 188. 190. 189. 191. 186. 189. 191. 189.
 188. 190. 188. 190. 189. 188. 189. 189. 187. 192. 190. 189. 188. 188.
 191. 189. 192. 190. 190. 189. 191. 189. 190. 189. 189. 188. 195. 189.
 192. 188. 189. 192. 189. 192. 190. 191. 193. 193. 189. 193. 189. 192.
 190. 191. 190. 191. 193. 195. 190. 188. 188. 190. 189. 193. 190. 190.
 188. 190. 192. 189. 189. 189. 189. 189. 190. 190. 188. 189. 189. 190.
 190. 191. 191. 193. 191. 193. 189. 192. 190. 192. 191. 191. 190. 190.
 193. 189. 190. 190. 195. 190. 190. 190. 190. 188. 188. 195. 192. 191.
 192. 

In [43]:
# Global thresholding
_, thresh1 = cv.threshold(img_gray,100,255,cv.THRESH_BINARY)
_, thresh2 = cv.threshold(img_gray,125,255,cv.THRESH_BINARY)
_, thresh3 = cv.threshold(img_gray,150,255,cv.THRESH_BINARY)


titles = ['Original Image','Grayscale Image','100','125','150']
images = [img, img_gray, thresh1, thresh2, thresh3]

viewer = napari.Viewer()
for i in range(len(images)):
    viewer.add_image(images[i], name=titles[i])

In [49]:
# Adapative thresholding
thm = cv.adaptiveThreshold(img_gray,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY,31,4)
thg = cv.adaptiveThreshold(img_gray,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY,31,4)

titles = ['Original Image','Grayscale Image','Adaptive Guassian','Adaptive mean']
images = [img, img_gray, thm, thg]

viewer = napari.Viewer()
for i in range(len(images)):
    viewer.add_image(images[i], name=titles[i])

In [55]:
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(4,4))
closing = cv.morphologyEx(thm, cv.MORPH_CLOSE, kernel)

viewer = napari.Viewer()
viewer.add_image(img, name='Original Image')
viewer.add_image(closing, name='closing')

<Image layer 'opening' at 0x27718573fd0>

In [72]:
closing = cv.morphologyEx(thresh2, cv.MORPH_CLOSE, kernel)
num,labels,stats,centroids = cv.connectedComponentsWithStats(closing,connectivity=8)
for i in range(num):
    print(np.sum(labels==i),stats[i][4])


6942 6942
779482 779482
1 1
2 2
5 5
